# Trajectory analysis
Contact matrix frame-wise

In [1]:
# Queste sono da importare direttamente dal pdb ma c'e' da risolvere la numerazione

amino_dict_num = {0:'TYR_1', 1:'THR_2', 2:'ILE_3', 3:'ALA_4', 4:'ALA_5', 5:'LEU_6', 6:'LEU_7', 7:'SER_8', 8:'PRO_9', 9:'TYR_10', 10:'SER_11'}
pdb_resids = list(range(0, 11))

#ref_structure = fibril.pdb
#traj = start_clust-dt10000.xtc
Min_Distance = 4


nchains = 2000
concentration = 13
replica_number = 1
atoms_in_chain = 85

In [2]:
%cd make_fibril_2000_13mM_300K_1/analysis_test
%ls

/media/data/emanuele/TTR/greta_test_cutoff_55_ex_3/epsilon_0295_c12_Native2reweight_noN1_gd0/concentration_gradient/make_fibril_2000_13mM_300K_1/analysis_test
'#fibril.pdb.1#'                    fibril_compare.pdb
'#fibril.pdb.2#'                    histo-clust.xvg
'#fibril.pdb.3#'                    last_frame.ndx
'#fibril.pdb.4#'                    last_frame.xtc
'#fibril.pdb.5#'                    matrix-dt10000.dat
'#fibril.pdb.6#'                    maxclust.ndx
'#fibril.pdb.7#'                    maxclust.xvg
 13mM_histo_dt-10000.png            nclust.xvg
 13mM_matrix_dt-10000.png           plot_histo_norm_dt10000.txt
 2000_pep-13mM.gro@                 plot_histogram_dt10000.txt
 NOPBC_prod_2000-13mM-dt1000.xtc@   plot_matrix_dt10000.txt
 avclust.xvg                        prod_2000-13mM-dt10000.xtc
 csize.xpm                          prod_2000-13mM.tpr@
 csizew.xpm                         temp.xvg
 fibril.pdb


In [3]:
import numpy as np
import MDAnalysis as mda
from MDAnalysis.analysis.distances import distance_array
import networkx as nx
import pandas as pd
import mdtraj as md
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("TkAgg")
import itertools
import time
import os, os.path



def make_plots(matrix):
    #print(matrix_dict)

    # Changing the index so now it is in picoseconds
    #matrix[0] = matrix[0].apply(lambda x:x*10)
    #matrix = matrix.set_index([0])

    ### Data handling
    #### Defining the Maxclust and the number of frames
    clustsize = list(range(0, len(matrix.columns), 1))
    frames = list(range(0, len(matrix.index), 1))
    fibril = list(range(2, len(matrix.columns)))
    #not_fibril = list(range(3, 10, 1))


    #### Multiply the columns for its clustsize
    size = pd.Series(clustsize)
    matrix = matrix.mul(size, axis = 1)

    #### Replace 0 with n.an
    matrix[matrix == 0] = np.nan

    #### I don't care about monomers
    matrix = matrix.drop([0, 1], axis = 1)

    ### Make the histogram to compare with lab results
    # Set the not-fibril to 0 and have a sort of lag time 

# At the moment i keep also the matrix for multiple fibrils but now i focus on the normalized histogram

    histo_norm = matrix.copy()
    histo_norm[[2, 3, 4, 5, 6, 7, 8, 9, 10]] = 0
    histo_norm = histo_norm[fibril].sum(axis=1)

    
    # Matrix transpose
    #matrix = matrix.T
    matrix = [] # To save some RAM

    return matrix, histo_norm


def contact_extraction(contact_list, contacts_histogram, pdb_resids):
    # Function to extract the informations from the pairs list
    for an in pdb_resids:
        # pdb_resids is the aminocid number
        # Selection of the rows contaning the aminoacid number of interest.
        # Note the | and not the &
        is_an = (contact_list['ai_resnumber'] == an) | (contact_list['aj_resnumber'] == an)

        # Filtering a dataframe containing only the aminoacid selected.
        contacts_an = contact_list[is_an]
        #print(pairs_an.to_string())

        print(f'\nTotal amount of contacts made by {amino_dict_num.get(an)}:', len(contacts_an))    
        contacts_an.loc[contacts_an['ai_resnumber'] == an, 'an_with_who'] = contacts_an['aj_resname']# + '_' + contacts_an['aj_resnumber'].astype(str)
        contacts_an.loc[contacts_an['ai_resnumber'] != an, 'an_with_who'] = contacts_an['ai_resname']# + '_' + contacts_an['ai_resnumber'].astype(str)    
        #print(pairs_an.to_string())
        
        # Create Pandas Series with contact counts for every aminoacid in an selection:
        # an_with_who count of the unique values in column "an_with_who" in pairs_an dataframe
        an_with_who = contacts_an.an_with_who.value_counts()
        print(f'\nList of contacts based on aminoacid:\n{an_with_who.to_string()}')

        # Change "an_with_who" in a dictionary to put in a matrix
        contacts_histogram['{0}'.format(an)] = an_with_who.to_dict()
        #print(f'\nAnd the consequent nested dictionary:\n{contacts_histogram}\n')
    
    return contacts_histogram#, sigma_histogram

def make_contacts_histo_df(contacts_histogram):
    # Contact dataframe preparation
    histo_df = pd.DataFrame(contacts_histogram)
    histo_df[''] = histo_df.index.astype(str)
    histo_df[''] = histo_df[''].str.split('_').str[1].astype(int)
    histo_df.sort_values(by = [''], inplace = True)
    histo_df.drop(columns='', inplace=True)
    histo_df = histo_df.fillna(0)
    histo_df.columns = list(amino_dict_num.values())
    #print(histo_df.to_string())
    return histo_df

def rename_chains(filename):
    with open('fibril.pdb', 'r+') as f:
        # Read the pdb file
        lines = f.readlines()
        # Change the column number 22 which is the chain id in the pdb
        column = 22 - 1

        # We want to change all the lines of the atoms
        replace_line = list(range(4,(len(lines)-2)))

        # We start from a chain id 0 (to 9, stated below)
        # This is possibile since mdtraj does not rebuild the "broken chains"
        chain_id = 0
        count_atoms = 1
        # Delete the character for every line in range and replace with our chain id
        for l in replace_line:
            lines[l]=lines[l][0:column]+str(chain_id)+lines[l][column+1:]
            #print(count_atoms, 'count_atoms')
            count_atoms += 1
            #print(chain_id, 'word')
            #print(l, 'line')
            if count_atoms > atoms_in_chain: # as soon as we complete the first molecule
                chain_id += 1 # We add one chain identifier
                count_atoms = 1 # Reset the atom counter
                if chain_id == 10: # Only the column 22 so only one digit is allowed, therefore we start again
                    chain_id = 0

        # Delete the pdb file
        f.seek(0)

        # Replace the pdb with the new lines
        for i in lines:
            f.write(i)

In [4]:
# Clustsize information
dat_file = list(filter(lambda k: '.dat' in k, os.listdir('.')))
# From a single element list to a string
dat_file = ''.join(dat_file)

# Here the matrix is made like the aggregation plto to know which initial frames we should skip
matrix, histo_norm = make_plots(pd.read_csv(dat_file, sep = '\\s+', header = None))
# From pandas series to dataframe and count how many 0 are present in the df
histo_norm = histo_norm.to_frame()

print(histo_norm)
# How many frames we want to skip
frames_to_skip = (~histo_norm.all(1)).sum()
frames_to_skip

         0
0      0.0
1      0.0
2      0.0
3     13.0
4     20.0
..     ...
76  1961.0
77  1964.0
78  1964.0
79  1963.0
80  1962.0

[81 rows x 1 columns]


3

In [11]:
# Trajectory parameters

trajectory_file = list(filter(lambda k: '.xtc' in k, os.listdir('.')))
trajectory_file = list(filter(lambda k: 'NOPBC' not in k, trajectory_file))

# Me lo sono trovato e non so che cavolo sia
trajectory_file = list(filter(lambda k: '.npz' not in k, trajectory_file))

trajectory_file.remove('last_frame.xtc')
print(trajectory_file)
trajectory_file = ''.join(trajectory_file)
print(trajectory_file)
topology = md.load('fibril.pdb').topology
trajectory = md.load(trajectory_file, top = 'fibril.pdb')
#trajectory = md.load(trajectory_file, top = f'{nchains}_pep-{concentration}mM_{replica_number}.gro')

trajectory = trajectory[frames_to_skip:]
print(trajectory)

frames=trajectory.n_frames				#Number of frames da provare a vedere cosa viene fuori con MDAnalysis
chains=topology.n_chains				#Number of chains
atoms=int(topology.n_atoms/chains)			#Number of atoms in each monomer 
AminoAcids = int(topology.n_residues/chains)#-2		#Number of residues per chain ('-2' avoid the N- and C- cap residues as individual residues)


print(f'Number of chains in the system: {chains}\nNumber of atoms in each monomer:{atoms}\nNumber of aminoacids in every chain:{AminoAcids}')

# Create Universe
print('Create Universe')
uni = mda.Universe('fibril.pdb',trajectory_file)
n,t = list(enumerate(uni.trajectory))[0]
box = t.dimensions[:6]
print('Box dimensions', box)

['prod_2000-13mM-dt10000.xtc']
prod_2000-13mM-dt10000.xtc
<mdtraj.Trajectory with 78 frames, 170000 atoms, 22000 residues, and unitcells>
Number of chains in the system: 2000
Number of atoms in each monomer:85
Number of aminoacids in every chain:11
Create Universe
Box dimensions [630. 630. 630.  90.  90.  90.]


In [12]:
# Questa la avremo gia


#string = f'gmx_mpi editconf -f prod_{nchains}-{concentration}mM.tpr -o fibril.pdb'
#string = f'gmx_mpi editconf -f prod_{nchains}-{concentration}mM_{replica_number}.tpr -o fibril.pdb'
#!{string}

#rename_chains('fibril.pdb')

In [13]:
isum=1
atoms_list, atomsperAminoAcid, residue_list=[], [], []
for residue in topology.chain(0).residues:
    atoms_list.append(residue.n_atoms)
    residue_list.append(residue)
    ', '.join(map(lambda x: "'" + x + "'", str(residue_list)))
print(f'{len(residue_list)} residues in chain:{residue_list}')

#The N- and C- cap residues are part of the 1st and last residue index. If no N- and C- cap residues for the protein, comment the line below using "#"
#del residue_list[0]; del residue_list[-1] 

for ii in range(len(atoms_list)):
    isum = isum + atoms_list[ii]
    atomsperAminoAcid.append(isum)
atomsperAminoAcid.insert(0, 1)

#print('atomsperAminoAcid', atomsperAminoAcid) # Questo e' perche' le chains partono da 0 e i gruppi vengono ottenuti moltiplicando per il resid

#The N- and C- cap residues are part of the 1st and last residue index. If no N- and C- cap residues for the protein, comment the line below using "#"
#del atomsperAminoAcid[1]; del atomsperAminoAcid[-2]      

atom_Groups = [[] for x in range(chains)]
m_start=0
for m in range(0,chains):
    # With this functions every chain is defined as a group of atoms
    # m is the chain number, atom is
    #  the number of atoms in every chain
    m_end = atoms * (m+1)
    atom_Groups[m].extend([uni.select_atoms('bynum '+ str(m_start) + ':' + str(m_end))])
    m_start = m_end + 1

AtomGroups = [[] for x in range(chains)]
for m in range(0,chains):
    for aa in range(0,AminoAcids):
        AtomGroups[m].extend([uni.select_atoms('bynum '+str(atoms*m + atomsperAminoAcid[aa])+':'+str(atoms*m + atomsperAminoAcid[aa + 1] - 1 ))])

TYR1
THR2
ILE3
ALA4
ALA5
LEU6
LEU7
SER8
PRO9
TYR10
SER11
11 residues in chain:[TYR1, THR2, ILE3, ALA4, ALA5, LEU6, LEU7, SER8, PRO9, TYR10, SER11]


In [ ]:
cluster_for_traj, chain_resid_traj, contacts_traj, norm_contacts_traj = {}, {}, {}, {}

start_frame_time = time.time()
for tt in uni.trajectory[frames_to_skip:]:
    '''
    Analysis of chains interactions in every frame of trajectory.
    In every frame it is created a Dataframe containing a list of
    interactions among the chains involved in a fibril (less than 4A)
    '''

    start_time = time.time()
    print(f'\n\nStarting frame {tt.frame+1} analysis out of {len(uni.trajectory)} total frames')
    
    # Creation of an empty dataframe which will be filled with a list of 
    # interactions between chains less than 4A far

    chain_list = []
    for i in range(chains):
        # Here a combinations of chains is made for every chain in the system
        chain_list.append(i)
    chain_comb = list(itertools.combinations(chain_list, 2))


    df_chain_ai, df_chain_aj, chain_distance = [], [], []
    for n in range(0, len(chain_comb)):
        # Here the combination list of tuple is rearranged in two lists
        # Only the chains within 4A cutoff are inserted into the dataframe
        i = chain_comb[n][0]
        j = chain_comb[n][1]
        dist = distance_array(atom_Groups[i][0].positions,atom_Groups[j][0].positions,box).min()
        if dist <= Min_Distance:
            df_chain_ai.append(i)
            df_chain_aj.append(j)
            chain_distance.append(dist)
    
    # The following DataFrame contains all the interactions among the chains involved in fibril
    chains_interactions = pd.DataFrame(columns=['chain_ai', 'chain_aj', 'min_distance'])
    chains_interactions['chain_ai'] = df_chain_ai
    chains_interactions['chain_aj'] = df_chain_aj
    chains_interactions['min_distance'] = chain_distance
    
    
    chains_in_fibril = list((set(chains_interactions['chain_ai'])) | (set(chains_interactions['chain_aj'])))
    chains_in_fibril.sort()
    
    print("Counted chains in fibril in %s seconds\n" % (time.time() - start_time))
    print('Number of chains interactions:', len(chains_interactions))
    print(f'{len(chains_in_fibril)} chains in fibril:{chains_in_fibril}')


    if len(chains_interactions) != 0:
        # Here a DataFrame of the interacting aminoacids is made for every contacting chain
        trj_amino_interactions = []
        for i, j in zip(df_chain_ai, df_chain_aj):
            # For every pair of chains
            en_i, atoms_i, ai_resnumber = [], [], []
            for e, a in enumerate(AtomGroups[int(float(i))]):
                # Select the AtomGroup of the wanted chain (i)
                # Append the residue number of the chain + the chain number
                en_i.append(str(e) + '_' + str(i))
                # Append the residue number alone
                ai_resnumber.append(e)
                # Append the corresponding AtomGroups of the chain
                atoms_i.append(a)
            en_j, atoms_j, aj_resnumber = [], [], []
            for e, a in enumerate(AtomGroups[int(float(j))]):
                # Select the AtomGroup of the wanted chain (i)
                # Append the residue number of the chain + the chain number
                en_j.append(str(e) + '_' + str(j))
                # Append the residue number alone
                aj_resnumber.append(e)
                # Append the corresponding AtomGroups of the chain
                atoms_j.append(a)

            # Those three lists have the same order, so combination will have the same shape
            # 11 aminoacids per chain -> 121 combinations
            en_comb = list(itertools.product(en_i, en_j)) # len 121
            resnumber_comb = list(itertools.product(ai_resnumber, aj_resnumber))
            atoms_comb = list(itertools.product(atoms_i, atoms_j))

            amino_distance = []
            for atoms in atoms_comb:            
                amino_distance.append(distance_array(atoms[0].positions,atoms[1].positions,box).min())

            # Build the consequent dataframe
            amino_interactions = pd.DataFrame(columns=['resid_chain_i', 'resid_chain_j', 'ai_resnumber', 'aj_resnumber', 'ai_resname', 'aj_resname', 'amino_distance'])
            amino_interactions['resid_chain_i'] = [x[0] for x in en_comb]
            amino_interactions['resid_chain_j'] = [x[1] for x in en_comb]
            amino_interactions['ai_resnumber'] = [x[0] for x in resnumber_comb]
            amino_interactions['aj_resnumber'] = [x[1] for x in resnumber_comb]
            amino_interactions['ai_resname'] = amino_interactions['ai_resnumber'].map(amino_dict_num)
            amino_interactions['aj_resname'] = amino_interactions['aj_resnumber'].map(amino_dict_num)
            amino_interactions['amino_distance'] = amino_distance
            # Append as a list of DataFrames
            trj_amino_interactions.append(amino_interactions)

        # Merge the Dataframes in a single one
        trj_amino_interactions = pd.concat(trj_amino_interactions, ignore_index=True)
        # Keep only the contacts below the cutoff of 4A
        trj_amino_interactions = trj_amino_interactions.loc[trj_amino_interactions['amino_distance'] < Min_Distance]
        
        # Creation of a contact map for every interacting aminoacid in a fibril
        contacts_histogram = {}
        frame_contacts_histogram = contact_extraction(trj_amino_interactions, contacts_histogram, pdb_resids) # Questo pdb resids alla fine e' un elenco degli aminoacidi da mettere con residue_list che hai gia'
        frame_contacts = make_contacts_histo_df(frame_contacts_histogram)
        
        # Creation of the same contact matrix but with normalized values
        contacts_norm = frame_contacts.div(frame_contacts.max().max())
        
        # Some roundings
        frame_contacts = frame_contacts.round(0).astype(int)
        contacts_norm = contacts_norm.round(2)

        # Save the DataFrames in a dictionary where the key is the number of frame
        # and the value are the Dataframes
        #cluster_for_traj[tt.frame+1] = chains_interactions
        #chain_resid_traj[tt.frame+1] = trj_amino_interactions
        contacts_traj[tt.frame+1] = frame_contacts
        norm_contacts_traj[tt.frame+1] = contacts_norm
        
        print("Frame %s completed in %s seconds ---" % (tt.frame+1, time.time() - start_time))




# FLAG if diverso da 0

#print(norm_contacts_traj)
print("Contact matrixes completed in %s seconds ---" % (time.time() - start_frame_time))

In [ ]:
#for k in chain_resid_traj:
#    print(f'Frame {k} out of {len(chain_resid_traj)} \n{chain_resid_traj[k].to_string()}\n\n\n\n')

In [ ]:
#pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_columns = None
pd.options.display.width = None
#for k in contacts_traj:
  #  print(f'Frame {k} out of {len(contacts_traj)} \n{contacts_traj[k].to_string()}\n\n\n\n')

In [ ]:
#pd.options.display.float_format = '{:.2f}'.format
#pd.options.display.max_columns = None
#for k in norm_contacts_traj:
#    print(f'Frame {k} out of {len(norm_contacts_traj)} \n{norm_contacts_traj[k].to_string()}\n\n\n\n')

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker
from matplotlib.colors import ListedColormap
import seaborn as sns
import os

os.mkdir('subplots')

counter = 1
for frame, contacts in contacts_traj.items():
    fig = plt.figure(figsize=(10,10))
    contact_matrix = contacts.to_numpy()
    #contact_matrix = contact_matrix.round(decimals=2)  
    g = sns.heatmap(contact_matrix, annot=True, cbar=False, cmap='Blues', fmt='d')
    plt.title(f'Frame {frame}')
    g.set_xticks(np.arange(len(contacts.columns))+0.5)
    g.set_yticks(np.arange(len(contacts.index))+0.5)
    g.set_xticklabels(list(contacts.columns), rotation=90, horizontalalignment='right', ha='center')
    g.set_yticklabels(list(contacts.index), rotation=0)
    g.xaxis.set_label_position('top')
    g.xaxis.set_ticks_position('top')
    sub_plot_fig = g.get_figure()
    sub_plot_fig.savefig(f'subplots/contacts_frame{frame}.png')
    counter = counter + 1
    plt.close()

fig = plt.figure(figsize=(60,60))
counter = 1
for frame, contacts in contacts_traj.items():
    contact_matrix = contacts.to_numpy()
    #contact_matrix = contact_matrix.round(decimals=2)  
    plt.subplot(6, 6, counter)
    g = sns.heatmap(contact_matrix, annot=True, cbar=False, cmap='Blues', fmt='d')
    plt.title(f'Frame {frame}')
    g.set_xticks(np.arange(len(contacts.columns))+0.5)
    g.set_yticks(np.arange(len(contacts.index))+0.5)
    g.set_xticklabels(list(contacts.columns), rotation=90, horizontalalignment='right', ha='center')
    g.set_yticklabels(list(contacts.index), rotation=0)
    g.xaxis.set_label_position('top')
    g.xaxis.set_ticks_position('top')
    counter = counter + 1
fig.savefig(f'contacts.png')


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker
from matplotlib.colors import ListedColormap
import seaborn as sns

counter = 1
for frame, contacts in norm_contacts_traj.items():
    fig = plt.figure(figsize=(10,10))
    contact_matrix = contacts.to_numpy()
    #contact_matrix = contact_matrix.round(decimals=2)  
    g = sns.heatmap(contact_matrix, annot=True, cbar=False, cmap='Blues', fmt='.2f')
    plt.title(f'Frame {frame}')
    g.set_xticks(np.arange(len(contacts.columns))+0.5)
    g.set_yticks(np.arange(len(contacts.index))+0.5)
    g.set_xticklabels(list(contacts.columns), rotation=90, horizontalalignment='right', ha='center')
    g.set_yticklabels(list(contacts.index), rotation=0)
    g.xaxis.set_label_position('top')
    g.xaxis.set_ticks_position('top')
    sub_plot_fig = g.get_figure()
    sub_plot_fig.savefig(f'subplots/contacts_norm_frame{frame}.png')
    counter = counter + 1
    plt.close()

fig = plt.figure(figsize=(60,60))
counter = 1
for frame, contacts in norm_contacts_traj.items():
    contact_matrix = contacts.to_numpy()
    #contact_matrix = contact_matrix.round(decimals=2)  
    plt.subplot(6, 6, counter)
    g = sns.heatmap(contact_matrix, annot=True, cbar=False, cmap='Blues', fmt='.2f')
    plt.title(f'Frame {frame}')
    g.set_xticks(np.arange(len(contacts.columns))+0.5)
    g.set_yticks(np.arange(len(contacts.index))+0.5)
    g.set_xticklabels(list(contacts.columns), rotation=90, horizontalalignment='right', ha='center')
    g.set_yticklabels(list(contacts.index), rotation=0)
    g.xaxis.set_label_position('top')
    g.xaxis.set_ticks_position('top')
    counter = counter + 1
fig.savefig(f'contacts_norm.png')


In [ ]:
string = f"convert -delay 50 -loop 0 $(for i in $(seq 1 36); do echo subplots/contacts_frame${{i}}.png; done) contacts_plots.gif"
!{string}
#!{'code contacts_plots.gif'}

In [ ]:
string = f"convert -delay 50 -loop 0 $(for i in $(seq 1 36); do echo subplots/contacts_norm_frame${{i}}.png; done) contacts_norm_plots.gif"
!{string}
#!{'code contacts_norm_plots.gif'}

In [ ]:
import nglview as nv

t = nv.MDAnalysisTrajectory(uni)
w = nv.NGLWidget(t)
w